In [7]:
import numpy as np 
import pandas as pd
import re

In [5]:
metaphlan = pd.read_csv("/home/bruce1996/data/MCI/metaphlan_result/metaphlan3_merge_table.txt",sep='\t',index_col=0)

In [16]:
idx = [bool(re.search("Bifido|lactobacillus",x)) for x in metaphlan.index]

In [17]:
probiotics_matrix = metaphlan.loc[idx,:]
probiotics_matrix.drop(["NCBI_tax_id"],axis=1,inplace=True)
probiotics_matrix.index.name = None
sample = []
for s in probiotics_matrix.columns :
    if "-" in s :
        sample.append(s.split("-")[0])
    else :
        sample.append(s.split("_")[0])
probiotics_matrix.columns = sample

In [25]:
probiotics_matrix.to_csv("/home/bruce1996/data/MCI/manuscript_v3/supplement/probiotics_only_diversity/probiotics_metaphlan.txt",sep='\t')

In [26]:
df = pd.read_csv("/home/bruce1996/data/MCI/manuscript_v3/supplement/probiotics_only_diversity/probiotics_unifrac_matrix.txt",sep='\t',index_col=0)

In [28]:
metadata = pd.read_csv("/home/bruce1996/data/MCI/manuscript_v3/normal/consensus/mixed_clustering_without_rogosae.txt",sep='\t',index_col=0)

In [30]:
np.unique(metadata['cluster'].values)

array([1, 2, 3, 4])